In [10]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
import re

In [2]:
folder_path = "/Users/addakinthomas/Desktop/Finances/amex/"
dfs = []

for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)

        month_name = file.split("_")[0]

        df = pd.read_csv(file_path)

        dfs.append(df)

final_df = pd.concat(dfs,ignore_index=True)
final_df['Date'] = pd.to_datetime(final_df['Date'], errors='coerce', format='%m/%d/%Y')

# Extract Month Name and Day
final_df['Month'] = final_df['Date'].dt.strftime('%B')  # "January"
final_df['Day'] = final_df['Date'].dt.day  # Extracts day as an integer (1-31)

# Drop the original 'Date' column
final_df = final_df.drop(columns=['Date'])

# Display final DataFrame
print(final_df)

                                    Description     Card Member  Account #  \
0                                          UBER  ADDAKIN THOMAS   -53004.0   
1                                          UBER  ADDAKIN THOMAS   -53004.0   
2    Interest Charge on Pay Over Time Purchases  ADDAKIN THOMAS   -53004.0   
3    TST* MIKE'S BEER BARPITTSBURGH          PA  ADDAKIN THOMAS   -53004.0   
4                                          UBER  ADDAKIN THOMAS   -53004.0   
..                                          ...             ...        ...   
165  AplPay STARBUCKS    800-782-7282        WA  ADDAKIN THOMAS   -53004.0   
166  THE MERCHANT KITCHENBOSTON              MA  ADDAKIN THOMAS   -53004.0   
167  TST* CRAFT FOOD HALLBOSTON              MA  ADDAKIN THOMAS   -53004.0   
168  TST* TATTE BAKERY ONBOSTON              MA  ADDAKIN THOMAS   -53004.0   
169  MA MAISON 6500000034BOSTON              MA  ADDAKIN THOMAS   -53004.0   

     Amount                                   Extended Details 

In [3]:
df = final_df.drop(columns = ['Card Member', 'Account #', 'Zip Code', 'Reference', 'Address', 'Country', 'Extended Details'])
print(df)

                                    Description  Amount  \
0                                          UBER   16.39   
1                                          UBER   20.93   
2    Interest Charge on Pay Over Time Purchases   43.88   
3    TST* MIKE'S BEER BARPITTSBURGH          PA    3.67   
4                                          UBER   14.96   
..                                          ...     ...   
165  AplPay STARBUCKS    800-782-7282        WA   10.00   
166  THE MERCHANT KITCHENBOSTON              MA   53.50   
167  TST* CRAFT FOOD HALLBOSTON              MA   19.14   
168  TST* TATTE BAKERY ONBOSTON              MA   14.04   
169  MA MAISON 6500000034BOSTON              MA   85.00   

                   Appears On Your Statement As         City/State  \
0    AplPay Uber Trip    help.uber.com       CA  SAN FRANCISCO\nCA   
1    AplPay Uber Trip    help.uber.com       CA  SAN FRANCISCO\nCA   
2    Interest Charge on Pay Over Time Purchases                NaN   
3    TST* M

In [4]:
# Define the parent category mapping
category_mapping = {
    'Transportation-Taxis & Coach': 'Transportation',
    'Transportation-Fuel': 'Transportation',
    'Transportation-Parking Charges': 'Transportation',
    'Travel-Airline': 'Travel',
    'Travel-Vehicle Rental': 'Travel',
    'Restaurant-Bar & Café': 'Food & Drinks',
    'Restaurant-Restaurant': 'Food & Drinks',
    'Merchandise & Supplies-Wholesale Stores': 'Shopping',
    'Merchandise & Supplies-General Retail': 'Shopping',
    'Merchandise & Supplies-Internet Purchase': 'Shopping',
    'Merchandise & Supplies-Book Stores': 'Shopping',
    'Merchandise & Supplies-Groceries': 'Groceries',
    'Merchandise & Supplies-Computer Supplies': 'Technology & Supplies',
    'Entertainment-General Attractions': 'Entertainment & Leisure',
    'Entertainment-Associations': 'Entertainment & Leisure',
    'Fees & Adjustments-Fees & Adjustments': 'Fees & Adjustments',
    'Other-Government Services': 'Government & Services',
    'Other-Miscellaneous': 'Miscellaneous'
}

# Add the new 'Parent Category' column to the DataFrame
df['Parent Category'] = df['Category'].map(category_mapping)
df['Account'] = 'AMEX'
df = df[df['Description'] != "MOBILE PAYMENT - THANK YOU"]

# Replace all commas with spaces in all string columns
df = df.map(lambda x: x.replace(",", " ") if isinstance(x, str) else x)

# Display the first few rows
print(df)

                                    Description  Amount  \
0                                          UBER   16.39   
1                                          UBER   20.93   
2    Interest Charge on Pay Over Time Purchases   43.88   
3    TST* MIKE'S BEER BARPITTSBURGH          PA    3.67   
4                                          UBER   14.96   
..                                          ...     ...   
165  AplPay STARBUCKS    800-782-7282        WA   10.00   
166  THE MERCHANT KITCHENBOSTON              MA   53.50   
167  TST* CRAFT FOOD HALLBOSTON              MA   19.14   
168  TST* TATTE BAKERY ONBOSTON              MA   14.04   
169  MA MAISON 6500000034BOSTON              MA   85.00   

                   Appears On Your Statement As         City/State  \
0    AplPay Uber Trip    help.uber.com       CA  SAN FRANCISCO\nCA   
1    AplPay Uber Trip    help.uber.com       CA  SAN FRANCISCO\nCA   
2    Interest Charge on Pay Over Time Purchases                NaN   
3    TST* M

In [5]:
# Define the folder path inside your Desktop
folder_path = r"/Users/addakinthomas/Desktop/Finances/data"  # Update with your actual username

# Define the file path for the CSV
file_path = os.path.join(folder_path, "curated_finances.csv")

# Write the DataFrame to a CSV file inside the folder
df.to_csv(file_path, index=False)

# Confirmation message
print(f"CSV file saved successfully at: {file_path}")

CSV file saved successfully at: /Users/addakinthomas/Desktop/Finances/data/curated_finances.csv


In [6]:
pip install pdfplumber pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pdfplumber
import re
import pandas as pd

# Define the path to the PDF
pdf_path = "/Users/addakinthomas/Desktop/Finances/chase/january_2025.pdf"

# Extract text from the PDF
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[1]  # Adjust index if needed
    text = page.extract_text()  # Extract the full text

# Print the extracted text to verify
print(text)

# Ensure text is not None before proceeding
if text:
    # Regular expression to match transaction entries
    pattern = re.compile(
        r"(\d{2}/\d{2})\s+(.+?)\s+([-]?\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s+(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)"
    )

    # Extract transactions using regex
    transactions = pattern.findall(text)

    # Convert to DataFrame
    df = pd.DataFrame(transactions, columns=["Date", "Description", "Amount", "Balance"])

    # Convert Amount and Balance to numeric values
    df["Amount"] = df["Amount"].str.replace(",", "").astype(float)
    df["Balance"] = df["Balance"].str.replace(",", "").astype(float)

    # Display the DataFrame
    print(df)

else:
    print("No text extracted from the PDF.")


December 19, 2024throughJanuary 21, 2025
Account Number:
000000727816057
*start*transaction detail
TRANSACTION DETAIL
DATE DESCRIPTION AMOUNT BALANCE
Beginning Balance $1,751.91
12/19 American Express ACH Pmt M8014 Web ID: 2005032111 -23.20 1,728.71
12/23 Remote Online Deposit 1 25.00 1,753.71
12/23 Venmo Payment 1039105423155 Web ID: 3264681992 -40.00 1,713.71
12/23 Venmo Payment 1039105425873 Web ID: 3264681992 -20.00 1,693.71
12/23 Card Purchase 12/21 Draftkings 6179866744 MA Card 1363 -10.00 1,683.71
12/23 Card Purchase 12/21 Draftkings 6179866744 MA Card 1363 -10.00 1,673.71
12/24 Prosper Marketpl Ibi 2024 PPD ID: ACH8446990 -164.81 1,508.90
12/26 Real Time Transfer Recd From Aba/Contr Bnk-021000021 From: 49.13 1,558.03
Bnf-Venmo Ref: 2412251330781298 Info: Text- Iid:
20241225021000021P1Brjpc01680021263 Recd: 08:59:22 Trn:
0192180360Gd
12/26 Payment Received 12/26 Draftkings Visa Direct FL Card 1363 402.81 1,960.84
12/26 Payment Received 12/26 Draftkings Visa Direct FL Card 1363 3

In [8]:
import pdfplumber
import re
import pandas as pd
import os

# Define the folder containing the PDF files
folder_path = "/Users/addakinthomas/Desktop/Finances/chase/"

# Initialize an empty list to store all transactions across multiple PDFs
all_transactions = []

# Loop through all PDF files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):  # Ensure we only process PDF files
        pdf_path = os.path.join(folder_path, filename)
        print(f"Processing: {filename}")

        # Open the PDF
        with pdfplumber.open(pdf_path) as pdf:
            num_pages = len(pdf.pages)  # Get total number of pages
            start_page = 1  # Page index for second page (0-based index)
            end_page = num_pages - 1  # 2nd to last page

            for page_num in range(start_page, end_page):  # Loop through selected pages
                page = pdf.pages[page_num]
                text = page.extract_text()

                if text:
                    # **Remove extra text before and after the transaction list**
                    text = re.sub(r".*?\*start\*transaction detail", "", text, flags=re.DOTALL)  # Remove everything before "*start*transaction detail"
                    text = re.sub(r"\*end\*transac.*", "", text, flags=re.DOTALL)  # Remove everything after "*end*transac"
                    text = text.strip()  # Clean up leading/trailing whitespace

                    # Regular expression to match transaction entries
                    pattern = re.compile(
                        r"(\d{2}/\d{2})\s+(.+?)\s+([-]?\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s+(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)"
                    )

                    # Extract transactions using regex
                    transactions = pattern.findall(text)

                    # Append transactions to the list, adding the file name for reference
                    for transaction in transactions:
                        all_transactions.append((filename,) + transaction)

# Convert extracted data into a DataFrame
df = pd.DataFrame(all_transactions, columns=["File", "Date", "Description", "Amount", "Balance"])

# Convert Amount and Balance to numeric values
df["Amount"] = df["Amount"].str.replace(",", "").astype(float)
df["Balance"] = df["Balance"].str.replace(",", "").astype(float)

# Display the DataFrame
print(df)

df.to_csv(os.path.join(folder_path, "chase_all_transactions.csv"), index=False) 

Processing: january_2025.pdf
Processing: february_2025.pdf
                 File   Date  \
0    january_2025.pdf  12/19   
1    january_2025.pdf  12/23   
2    january_2025.pdf  12/23   
3    january_2025.pdf  12/23   
4    january_2025.pdf  12/23   
..                ...    ...   
90  february_2025.pdf  02/18   
91  february_2025.pdf  02/18   
92  february_2025.pdf  02/19   
93  february_2025.pdf  02/20   
94  february_2025.pdf  02/20   

                                          Description   Amount  Balance  
0   American Express ACH Pmt M8014 Web ID: 2005032111   -23.20  1728.71  
1                               Remote Online Deposit     1.00    25.00  
2      Venmo Payment 1039105423155 Web ID: 3264681992   -40.00  1713.71  
3      Venmo Payment 1039105425873 Web ID: 3264681992   -20.00  1693.71  
4   Card Purchase 12/21 Draftkings 6179866744 MA C...   -10.00  1683.71  
..                                                ...      ...      ...  
90  Card Purchase 02/16 Betrivers PA 8

In [9]:
import pandas as pd
import os
import calendar

# File paths
chase_transactions_path = "/Users/addakinthomas/Desktop/Finances/chase/chase_all_transactions.csv"
curated_finances_path = "/Users/addakinthomas/Desktop/Finances/data/curated_finances.csv"

# Load the Chase transactions CSV
chase_df = pd.read_csv(chase_transactions_path)

# Extract numeric month and day from the 'Date' column (assuming MM/DD format)
chase_df["Month"] = chase_df["Date"].apply(lambda x: int(x.split("/")[0]))  # Convert month to integer
chase_df["Day"] = chase_df["Date"].apply(lambda x: int(x.split("/")[1]))  # Convert day to integer

# Convert numeric month to actual month name
chase_df["Month"] = chase_df["Month"].apply(lambda x: calendar.month_name[x])  # Convert to full month name

# Define a function to map categories based on the description
def map_category(description):
    description = description.lower()  # Normalize case for comparison
    
    # Example logic - customize as needed
    if "uber" in description or "lyft" in description:
        return "Transportation-Taxis & Coach", "Transportation"
    elif "grocery" in description or "walmart" in description or "costco" in description:
        return "Groceries", "Food & Drinks"
    elif "restaurant" in description or "bar" in description or "cafe" in description:
        return "Restaurant-Bar & Café", "Food & Drinks"
    elif "venmo" in description or "paypal" in description:
        return "Peer-to-Peer Payment", "Transfers"
    elif "amazon" in description or "best buy" in description:
        return "Online Shopping", "Shopping"
    elif "spotify" in description or "netflix" in description:
        return "Entertainment-Streaming", "Entertainment"
    elif "airline" in description or "delta" in description or "united" in description:
        return "Travel-Airline", "Travel"
    elif "hotel" in description or "airbnb" in description:
        return "Travel-Hotels", "Travel"
    elif "credit card payment" in description or "payment sent" in description:
        return "Credit Card Payment", "Debt Repayment"
    else:
        return "Other", "Other"  # Default category

# Apply the category mapping function
chase_df[["Category", "Parent Category"]] = chase_df["Description"].apply(
    lambda x: pd.Series(map_category(str(x)))
)

# Map the columns to match curated_finances.csv structure
mapped_df = pd.DataFrame({
    "Description": chase_df["Description"],
    "Amount": chase_df["Amount"],
    "Appears On Your Statement As": chase_df["Description"],  # Placeholder (same as Description)
    "City/State": "",  # No location info in Chase data
    "Category": chase_df["Category"],
    "Month": chase_df["Month"],  # Now contains full month names
    "Day": chase_df["Day"],
    "Parent Category": chase_df["Parent Category"],
    "Account": "Chase"  # Since all these transactions are from Chase
})

# Load existing curated finances data
curated_finances_df = pd.read_csv(curated_finances_path)

# Append new data to curated finances
updated_finances_df = pd.concat([curated_finances_df, mapped_df], ignore_index=True)

# Save the updated curated finances table
updated_finances_df.to_csv(curated_finances_path, index=False)

# Display success message
print(f"Updated curated finances saved to: {curated_finances_path}")


Updated curated finances saved to: /Users/addakinthomas/Desktop/Finances/data/curated_finances.csv
